In [1]:
import os 
import sys

In [2]:
!ls

README.md         evaluate.py       model.py          ptuning_script.py
__pycache__       imgs              predict.py        rv0.ipynb
data.py           label_normalized  ptuning.py


In [3]:
from ptuning_script import *

INFO 2022-10-18 15:10:36,701 utils.py:147] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [9]:
args = reformat_parser_tuple

In [12]:
d = dict(reformat_parser_df.values.tolist())
d["task_name"] = "bustm"
args_parser_namedtuple = namedtuple("args_config", d)
args = args_parser_namedtuple(**d)
args.task_name

'bustm'

In [13]:
label_normalize_json = os.path.join("./label_normalized",
                                        args.task_name + ".json")

In [14]:
label_norm_dict = None
with open(label_normalize_json, 'r', encoding="utf-8") as f:
    label_norm_dict = json.load(f)

convert_example_fn = convert_example if args.task_name != "chid" else convert_chid_example
evaluate_fn = do_evaluate if args.task_name != "chid" else do_evaluate_chid

In [16]:
train_ds, dev_ds, public_test_ds = load_dataset("fewclue",
                                                name=args.task_name,
                                                splits=("train_0", "dev_0",
                                                        "test_public"))

INFO 2022-10-18 15:17:39,384 download.py:117] unique_endpoints {''}
INFO 2022-10-18 15:17:39,385 download.py:117] unique_endpoints {''}
INFO 2022-10-18 15:17:39,386 download.py:250] Downloading fewclue_bustm.tar.gz from https://bj.bcebos.com/paddlenlp/datasets/FewCLUE/fewclue_bustm.tar.gz
100%|████████████████████████████████████████| 199/199 [00:01<00:00, 173.21it/s]
INFO 2022-10-18 15:17:41,771 download.py:269] File /Users/svjack/.paddlenlp/datasets/FewCLUE/fewclue_bustm.tar.gz md5 checking...
INFO 2022-10-18 15:17:41,775 download.py:287] Decompressing /Users/svjack/.paddlenlp/datasets/FewCLUE/fewclue_bustm.tar.gz...


In [19]:
###train_ds.__dict__

In [23]:
##pd.DataFrame(train_ds.__dict__["data"])

In [24]:
transform_fn = partial(transform_fn_dict[args.task_name],
                           label_normalize_dict=label_norm_dict)

In [25]:
train_ds = train_ds.map(transform_fn, lazy=False)
dev_ds = dev_ds.map(transform_fn, lazy=False)
public_test_ds = public_test_ds.map(transform_fn, lazy=False)

In [30]:
??transform_fn

In [29]:
###transform_fn_dict

In [34]:
train_ds[0]

{'id': 0, 'sentence1': '叫爸爸叫一声我听听', 'sentence2': '那你叫我一声爸爸', 'text_label': '很'}

In [37]:
model = ErnieForPretraining.from_pretrained('ernie-3.0-medium-zh')

[2022-10-18 16:04:20,693] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_medium_zh.pdparams and saved to /Users/svjack/.paddlenlp/models/ernie-3.0-medium-zh
[2022-10-18 16:04:20,695] [    INFO] - Downloading ernie_3.0_medium_zh.pdparams from https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_medium_zh.pdparams
100%|█████████████████████████████████████████| 313M/313M [08:25<00:00, 648kB/s]


In [38]:
tokenizer = AutoTokenizer.from_pretrained('ernie-3.0-medium-zh')

[2022-10-18 16:12:58,214] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'ernie-3.0-medium-zh'.
[2022-10-18 16:12:58,216] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_medium_zh_vocab.txt and saved to /Users/svjack/.paddlenlp/models/ernie-3.0-medium-zh
[2022-10-18 16:12:58,216] [    INFO] - Downloading ernie_3.0_medium_zh_vocab.txt from https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_medium_zh_vocab.txt
100%|█████████████████████████████████████████| 182k/182k [00:01<00:00, 118kB/s]
[2022-10-18 16:13:02,116] [    INFO] - tokenizer config file saved in /Users/svjack/.paddlenlp/models/ernie-3.0-medium-zh/tokenizer_config.json
[2022-10-18 16:13:02,117] [    INFO] - Special tokens file saved in /Users/svjack/.paddlenlp/models/ernie-3.0-medium-zh/special_tokens_map.json


In [40]:
json.dumps(train_ds[0])

'{"id": 0, "sentence1": "\\u53eb\\u7238\\u7238\\u53eb\\u4e00\\u58f0\\u6211\\u542c\\u542c", "sentence2": "\\u90a3\\u4f60\\u53eb\\u6211\\u4e00\\u58f0\\u7238\\u7238", "text_label": "\\u5f88"}'

In [44]:
train_ds[0]

{'id': 0, 'sentence1': '叫爸爸叫一声我听听', 'sentence2': '那你叫我一声爸爸', 'text_label': '很'}

In [41]:
src_ids, token_type_ids, mask_positions, mask_lm_labels = \
convert_example_fn(train_ds[0], tokenizer)

In [49]:
"".join(tokenizer.convert_ids_to_tokens(src_ids))

'[UNK][UNK][UNK][UNK][UNK][CLS][MASK]叫爸爸叫一声我听听[SEP]那你叫我一声爸爸[SEP]'

In [47]:
mask_positions

[6]

In [50]:
mask_lm_labels

[321]

In [51]:
"".join(tokenizer.convert_ids_to_tokens(mask_lm_labels))

'很'

In [53]:
####len(train_ds)

In [ ]:
#### sentence1 sentence2 text_label

In [1]:
import os
import sys

In [4]:
import argparse
import os
import sys
import random
import time
import json
from functools import partial

import numpy as np
import paddle
import paddle.nn.functional as F

import paddlenlp
from paddlenlp.transformers import AutoTokenizer
from paddlenlp.data import Stack, Tuple, Pad
from paddlenlp.datasets import load_dataset
from paddlenlp.transformers import LinearDecayWithWarmup

from model import ErnieForPretraining, ErnieMLMCriterion
from data import create_dataloader, transform_fn_dict
from data import convert_example, convert_chid_example
from evaluate import do_evaluate, do_evaluate_chid


In [12]:
from collections import namedtuple
import pandas as pd

In [7]:
parser = argparse.ArgumentParser()

parser.add_argument("--task_name", required=True, type=str, help="The task_name to be evaluated")
parser.add_argument("--p_embedding_num", type=int, default=1, help="number of p-embedding")
parser.add_argument("--batch_size", default=32, type=int, help="Batch size per GPU/CPU for training.")
parser.add_argument("--learning_rate", default=1e-5, type=float, help="The initial learning rate for Adam.")
parser.add_argument("--save_dir", default='./checkpoint', type=str, help="The output directory where the model checkpoints will be written.")
parser.add_argument("--max_seq_length", default=128, type=int, help="The maximum total input sequence length after tokenization. "
    "Sequences longer than this will be truncated, sequences shorter will be padded.")
parser.add_argument("--weight_decay", default=0.0, type=float, help="Weight decay if we apply some.")
parser.add_argument("--epochs", default=10, type=int, help="Total number of training epochs to perform.")
parser.add_argument("--warmup_proportion", default=0.0, type=float, help="Linear warmup proption over the training process.")
parser.add_argument("--init_from_ckpt", type=str, default=None, help="The path of checkpoint to be loaded.")
parser.add_argument("--seed", type=int, default=1000, help="random seed for initialization")
parser.add_argument('--device', choices=['cpu', 'gpu'], default="gpu", help="Select which device to train model, defaults to gpu.")
parser.add_argument('--save_steps', type=int, default=10000, help="Inteval steps to save checkpoint")
parser.add_argument("--rdrop_coef", default=0.0, type=float, help="The coefficient of KL-Divergence loss in R-Drop paper, for more detail please refer to https://arxiv.org/abs/2106.14448), if rdrop_coef > 0 then R-Drop works")

###args = parser.parse_args()

_StoreAction(option_strings=['--rdrop_coef'], dest='rdrop_coef', nargs=None, const=None, default=0.0, type=<class 'float'>, choices=None, help='The coefficient of KL-Divergence loss in R-Drop paper, for more detail please refer to https://arxiv.org/abs/2106.14448), if rdrop_coef > 0 then R-Drop works', metavar=None)

In [20]:
def test():
    exec("x = 0")
    return x

In [25]:
n_ext0 = namedtuple("arg0", {"s": 10})
n_ext0(**{"s": 1000}).s

1000

In [26]:
def parse_parser_add_arg(parser, as_named_tuple = False):
    args_df = pd.DataFrame(
    pd.Series(parser.__dict__["_actions"]).map(
    lambda x:x.__dict__
    ).values.tolist())
    args_df = args_df.explode("option_strings")
    args_df["option_strings"] = args_df["option_strings"].map(
    lambda x: x[2:] if x.startswith("--") else x
    ).map(
        lambda x: x[1:] if x.startswith("-") else x
        )
    args_df = args_df[["option_strings", "default"]]
    args = dict(args_df.values.tolist())
    if as_named_tuple:
        args_parser_namedtuple = namedtuple("args_config", args)
        return args_parser_namedtuple(**args)
    return args_df

In [27]:
parse_parser_add_arg(parser)

,option_strings,default
0,h,==SUPPRESS==
0,help,==SUPPRESS==
1,task_name,None
2,p_embedding_num,1
3,batch_size,32
4,learning_rate,0.00001
5,save_dir,./checkpoint
6,max_seq_length,128
7,weight_decay,0.0
8,epochs,10


In [30]:
parse_parser_add_arg(parser, as_named_tuple=True).rdrop_coef

0.0